## Vdelave

V našem prejšnjem primeru smo delali z visoko-dimenzionalnimi vektorji vreče besed dolžine `vocab_size`, pri čemer smo izrecno pretvarjali nizko-dimenzionalne vektorske predstavitve položajev v redke enovredne predstavitve. Ta enovredna predstavitev ni učinkovita glede pomnilnika, poleg tega pa se vsaka beseda obravnava neodvisno od drugih, tj. enovredni kodirani vektorji ne izražajo nobene semantične podobnosti med besedami.

V tej enoti bomo nadaljevali z raziskovanjem podatkovne zbirke **News AG**. Za začetek naložimo podatke in pridobimo nekaj definicij iz prejšnjega zvezka.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Kaj je vgrajevanje?

Ideja **vgrajevanja** je predstaviti besede z nižjedimenzionalnimi gostimi vektorji, ki nekako odražajo semantični pomen besede. Kasneje bomo razpravljali o tem, kako zgraditi smiselne vgrajene besede, vendar za zdaj razmišljajmo o vgrajevanju kot o načinu za zmanjšanje dimenzionalnosti vektorja besede.

Torej, vgrajevalna plast bi kot vhod vzela besedo in proizvedla izhodni vektor določene velikosti `embedding_size`. Na nek način je zelo podobna plasti `Linear`, vendar namesto da bi vzela enovročno kodiran vektor, bo lahko kot vhod vzela številko besede.

Z uporabo vgrajevalne plasti kot prve plasti v naši mreži lahko preklopimo iz modela vreče besed na model **vreče vgrajevanja**, kjer najprej vsako besedo v našem besedilu pretvorimo v ustrezno vgrajevanje, nato pa izračunamo neko agregatno funkcijo nad vsemi temi vgrajevanji, kot so `sum`, `average` ali `max`.

![Slika, ki prikazuje klasifikator vgrajevanja za pet zaporednih besed.](../../../../../translated_images/sl/embedding-classifier-example.b77f021a7ee67eee.webp)

Naša nevronska mreža klasifikatorja se bo začela z vgrajevalno plastjo, nato agregatno plastjo in linearnim klasifikatorjem na vrhu:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Obdelava spremenljive velikosti zaporedja

Zaradi te arhitekture je treba minibatche za našo mrežo ustvariti na določen način. V prejšnji enoti, ko smo uporabljali vrečo besed (bag-of-words), so imeli vsi BoW tenzorji v minibatchu enako velikost `vocab_size`, ne glede na dejansko dolžino našega besedilnega zaporedja. Ko preidemo na vektorske predstavitve besed (word embeddings), bomo imeli različno število besed v vsakem vzorcu besedila, in pri združevanju teh vzorcev v minibatche bomo morali uporabiti nekaj oblazinjenja (padding).

To lahko storimo z uporabo iste tehnike, kjer funkcijo `collate_fn` zagotovimo viru podatkov:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Usposabljanje klasifikatorja vdelav

Zdaj, ko smo definirali ustrezen nalagalnik podatkov, lahko model usposobimo z uporabo funkcije za usposabljanje, ki smo jo definirali v prejšnji enoti:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Opomba**: Tukaj treniramo samo za 25k zapisov (manj kot eno celotno epoho) zaradi prihranka časa, vendar lahko nadaljujete s treniranjem, napišete funkcijo za treniranje skozi več epoch in eksperimentirate s parametrom hitrosti učenja, da dosežete višjo natančnost. Morali bi doseči natančnost približno 90%.


### Plast EmbeddingBag in predstavitev zaporedij spremenljive dolžine

V prejšnji arhitekturi smo morali vsa zaporedja zapolniti do enake dolžine, da so ustrezala mini seriji. To ni najbolj učinkovit način za predstavitev zaporedij spremenljive dolžine – drugačen pristop bi bil uporaba **offset** vektorja, ki bi vseboval zamike vseh zaporedij, shranjenih v enem velikem vektorju.

![Slika, ki prikazuje predstavitev zaporedij z zamiki](../../../../../translated_images/sl/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Opomba**: Na zgornji sliki prikazujemo zaporedje znakov, vendar v našem primeru delamo z zaporedji besed. Kljub temu splošno načelo predstavitve zaporedij z offset vektorjem ostaja enako.

Za delo s predstavitvijo z zamiki uporabljamo plast [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Ta je podobna `Embedding`, vendar kot vhod prejme vsebinski vektor in offset vektor, poleg tega pa vključuje tudi plast za povprečenje, ki je lahko `mean`, `sum` ali `max`.

Tukaj je spremenjeno omrežje, ki uporablja `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Za pripravo nabora podatkov za učenje moramo zagotoviti pretvorbeno funkcijo, ki bo pripravila vektorski premik:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Upoštevajte, da za razliko od vseh prejšnjih primerov naša mreža zdaj sprejema dva parametra: podatkovni vektor in odmik vektor, ki sta različnih velikosti. Podobno nam naš nalagalnik podatkov zagotavlja 3 vrednosti namesto 2: tako besedilni kot odmikovni vektorji so zagotovljeni kot značilnosti. Zato moramo nekoliko prilagoditi našo funkcijo usposabljanja, da to upošteva:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantične vektorske predstavitve: Word2Vec

V našem prejšnjem primeru se je plast za vektorsko predstavitev v modelu naučila preslikati besede v vektorsko obliko, vendar ta predstavitev ni imela veliko semantičnega pomena. Bilo bi koristno, če bi se naučili takšne vektorske predstavitve, kjer bi bile podobne besede ali sinonimi predstavljeni z vektorji, ki so si blizu glede na neko vektorsko razdaljo (npr. evklidska razdalja).

Da to dosežemo, moramo naš model za vektorsko predstavitev predhodno naučiti na veliki zbirki besedil na specifičen način. Ena prvih metod za učenje semantičnih vektorskih predstavitev se imenuje [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Temelji na dveh glavnih arhitekturah, ki se uporabljata za ustvarjanje porazdeljene predstavitve besed:

 - **Neprekinjena vreča besed** (CBoW) — pri tej arhitekturi model učimo, da napove besedo iz okoliškega konteksta. Glede na n-gram $(W_{-2},W_{-1},W_0,W_1,W_2)$ je cilj modela napovedati $W_0$ na podlagi $(W_{-2},W_{-1},W_1,W_2)$.
 - **Neprekinjeni preskok-gram** (skip-gram) je nasproten CBoW. Model uporablja okoliško okno kontekstnih besed za napoved trenutne besede.

CBoW je hitrejši, medtem ko je skip-gram počasnejši, vendar bolje predstavlja redke besede.

![Slika, ki prikazuje algoritma CBoW in Skip-Gram za pretvorbo besed v vektorje.](../../../../../translated_images/sl/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Za eksperimentiranje z vektorsko predstavitvijo Word2Vec, predhodno naučeno na zbirki podatkov Google News, lahko uporabimo knjižnico **gensim**. Spodaj poiščemo besede, ki so najbolj podobne 'neural'.

> **Opomba:** Ko prvič ustvarjate vektorske predstavitve besed, lahko prenos podatkov traja nekaj časa!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Prav tako lahko izračunamo vektorske vdelave iz besede, ki jih uporabimo pri usposabljanju modela za klasifikacijo (za jasnost prikažemo le prvih 20 komponent vektorja):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Odlična stvar semantičnih vdelav je, da lahko manipulirate z vektorskim kodiranjem, da spremenite semantiko. Na primer, lahko zahtevamo, da najdemo besedo, katere vektorska predstavitev bi bila čim bližje besedama *kralj* in *ženska*, ter čim bolj oddaljena od besede *moški*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Tako CBoW kot Skip-Grams sta "prediktivni" vektorski predstavitvi, saj upoštevata le lokalne kontekste. Word2Vec ne izkorišča globalnega konteksta.

**FastText** nadgrajuje Word2Vec tako, da se nauči vektorske predstavitve za vsako besedo in za n-grame znakov, ki jih najdemo znotraj vsake besede. Vrednosti teh predstavitev se nato povprečijo v en vektor pri vsakem koraku učenja. Čeprav to doda veliko dodatnega računanja med predhodnim učenjem, omogoča, da vektorske predstavitve besed vključujejo informacije o podbesedah.

Druga metoda, **GloVe**, izkorišča idejo matrike so-pojavitev in uporablja nevronske metode za razgradnjo matrike so-pojavitev v bolj izrazne in nelinearne vektorske predstavitve besed.

Lahko se poigrate s primerom tako, da spremenite vektorske predstavitve v FastText in GloVe, saj gensim podpira več različnih modelov za vektorske predstavitve besed.


## Uporaba vnaprej naučenih vektorskih predstavitev v PyTorch

Primer zgoraj lahko prilagodimo tako, da matriko v naši slojni predstavitvi (embedding layer) predhodno napolnimo s semantičnimi vektorskimi predstavitvami, kot je Word2Vec. Upoštevati moramo, da se besedišča vnaprej naučenih predstavitev in našega besedilnega korpusa verjetno ne bodo ujemala, zato bomo uteži za manjkajoče besede inicializirali z naključnimi vrednostmi:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Zdaj trenirajmo naš model. Upoštevajte, da je čas, potreben za treniranje modela, bistveno daljši kot v prejšnjem primeru, zaradi večje velikosti vdelane plasti in posledično veliko večjega števila parametrov. Prav tako bomo morda morali model trenirati na več primerih, če želimo preprečiti prenaučenje.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

V našem primeru ne opazimo velikega povečanja natančnosti, kar je verjetno posledica precej različnih besedišč.  
Da bi premagali težavo različnih besedišč, lahko uporabimo eno od naslednjih rešitev:  
* Ponovno usposobimo model word2vec na našem besedišču  
* Naložimo naš nabor podatkov z besediščem iz že usposobljenega modela word2vec. Besedišče, ki se uporablja za nalaganje nabora podatkov, je mogoče določiti med nalaganjem.  

Drugi pristop se zdi lažji, še posebej zato, ker PyTorch `torchtext` ogrodje vsebuje vgrajeno podporo za vektorske predstavitve besed. Na primer, lahko ustvarimo besedišče, ki temelji na GloVe, na naslednji način:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Naloženi slovar ima naslednje osnovne operacije:
* Slovar `vocab.stoi` nam omogoča pretvorbo besede v njen indeks v slovarju
* `vocab.itos` opravlja nasprotno - pretvori številko v besedo
* `vocab.vectors` je matrika vektorskih predstavitev, tako da za pridobitev vektorske predstavitve besede `s` uporabimo `vocab.vectors[vocab.stoi[s]]`

Tukaj je primer manipulacije vektorskih predstavitev, ki prikazuje enačbo **kind-man+woman = queen** (moral sem nekoliko prilagoditi koeficient, da deluje):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Za učenje klasifikatorja z uporabo teh vdelav moramo najprej kodirati naš nabor podatkov z besediščem GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Kot smo videli zgoraj, so vsi vektorski vdelki shranjeni v matriki `vocab.vectors`. To omogoča zelo enostavno nalaganje teh uteži v uteži vdelane plasti z enostavnim kopiranjem:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Zdaj trenirajmo naš model in preverimo, ali dobimo boljše rezultate:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Eden od razlogov, zakaj ne opažamo znatnega povečanja natančnosti, je dejstvo, da nekaterih besed iz našega nabora podatkov ni v predhodno naučenem GloVe besedišču, zato so v bistvu prezrte. Da bi to premagali, lahko naučimo lastne vdelave na našem naboru podatkov.


## Kontekstualne vektorske predstavitve

Ena ključnih omejitev tradicionalnih vnaprej naučenih vektorskih predstavitev, kot je Word2Vec, je problem razločevanja pomenov besed. Čeprav lahko vnaprej naučene vektorske predstavitve zajamejo del pomena besed v kontekstu, je vsak možen pomen besede kodiran v isti vektor. To lahko povzroči težave v nadaljnjih modelih, saj imajo številne besede, kot je na primer beseda 'play', različne pomene glede na kontekst, v katerem so uporabljene.

Na primer, beseda 'play' ima v teh dveh stavkih precej različen pomen:
- Šel sem na **igro** v gledališče.
- John želi **igrati** s svojimi prijatelji.

Vnaprej naučene vektorske predstavitve zgoraj predstavljajo oba pomena besede 'play' v istem vektorju. Da bi premagali to omejitev, moramo graditi vektorske predstavitve na podlagi **jezikovnega modela**, ki je naučen na velikem korpusu besedil in *ve*, kako se besede lahko povezujejo v različnih kontekstih. Razprava o kontekstualnih vektorskih predstavitvah presega okvir tega vodiča, vendar se bomo k njim vrnili, ko bomo govorili o jezikovnih modelih v naslednji enoti.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
